In [ ]:
import streamlit as st
from langchain_community.llms import Ollama
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

# Set Streamlit page config
st.set_page_config(page_title="Local Chat with Mistral", layout="wide")

st.title("🧠 Local AI Chatbot using Ollama + LangChain")

# Initialize chat memory and model
@st.cache_resource
def load_conversation():
    llm = Ollama(model="mistral")  # Make sure Mistral is pulled with `ollama pull mistral`
    memory = ConversationBufferMemory()
    convo = ConversationChain(llm=llm, memory=memory)
    return convo

conversation = load_conversation()

# Chat interface
user_input = st.chat_input("Ask me anything...")

if user_input:
    with st.chat_message("user"):
        st.markdown(user_input)

    with st.chat_message("assistant"):
        response = conversation.run(user_input)
        st.markdown(response)


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from langchain_community.llms import Ollama
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
import pandas as pd
import PyPDF2
import io

# Init local model (make sure Mistral is pulled: `ollama pull mistral`)
llm = Ollama(model="mistral")

# Global file content
extracted_chunks = []

# Text splitter for long documents
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# File uploader widget
file_uploader = widgets.FileUpload(
    accept=".pdf,.xlsx,.xls", multiple=False, description="📁 Upload PDF or Excel"
)

# Input field + submit button
question_input = widgets.Text(
    placeholder="Ask a question about the file...",
    description="You:",
    layout=widgets.Layout(width="75%"),
)

submit_button = widgets.Button(
    description="Ask", button_style="success", layout=widgets.Layout(width="15%")
)

# Output area
output = widgets.Output()

# Display widgets
input_row = widgets.HBox([question_input, submit_button])
display(file_uploader, input_row, output)


# Function: extract file content and chunk it
def extract_file_content(change):
    global extracted_chunks
    extracted_chunks = []

    uploaded_files = file_uploader.value
    if not uploaded_files:
        return

    # Handle uploaded file
    file_info = uploaded_files[0]
    file_name = file_info["name"]
    file_type = file_name.split(".")[-1].lower()
    file_data = file_info["content"]

    # Process PDF
    if file_type == "pdf":
        reader = PyPDF2.PdfReader(io.BytesIO(file_data))
        text = "\n".join(
            page.extract_text() for page in reader.pages if page.extract_text()
        )
        docs = splitter.split_documents([Document(page_content=text)])

    # Process Excel
    elif file_type in ["xlsx", "xls"]:
        df = pd.read_excel(io.BytesIO(file_data))
        text = df.to_csv(index=False)
        docs = splitter.split_documents([Document(page_content=text)])

    else:
        docs = [Document(page_content="❌ Unsupported file type.")]

    extracted_chunks = docs
    with output:
        clear_output()
        print(f"✅ Loaded and processed file: {file_name} — {len(docs)} chunks ready.")
        print("📄 You can now ask questions about the document.")


# Function: handle question submission
def on_button_click(b):
    query = question_input.value
    if not extracted_chunks:
        with output:
            print("⚠️ Please upload a file first.")
        return
    if not query.strip():
        with output:
            print("⚠️ Please type a question.")
        return

    # Combine all chunks into one prompt (can be optimized later)
    full_text = "\n".join(chunk.page_content for chunk in extracted_chunks)
    prompt = (
        f"Given the following document:\n{full_text}\n\nAnswer this question:\n{query}"
    )
    response = llm(prompt)

    with output:
        print(f"\n🧠 Question: {query}")
        print(f"🤖 Answer: {response}")
    question_input.value = ""


# Bind event handlers
file_uploader.observe(extract_file_content, names="value")
submit_button.on_click(on_button_click)

In [14]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from langchain_community.llms import Ollama
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import pandas as pd
import PyPDF2
import faiss
import numpy as np
import io

# Local LLM (Ollama with mistral)
llm = Ollama(model="mistral")

# Embedder (local SentenceTransformers model)
embedder = SentenceTransformer("all-MiniLM-L6-v2")  # Small & fast

# Vector DB
index = None
chunk_map = []

# Splitter
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# Widgets
file_uploader = widgets.FileUpload(
    accept=".pdf,.xlsx,.xls", multiple=False, description="📁 Upload File"
)
question_input = widgets.Text(
    placeholder="Ask a question about the file...",
    description="You:",
    layout=widgets.Layout(width="75%"),
)
submit_button = widgets.Button(
    description="Ask", button_style="success", layout=widgets.Layout(width="15%")
)
output = widgets.Output()
input_row = widgets.HBox([question_input, submit_button])
display(file_uploader, input_row, output)

# --- FILE PROCESSING ---


def extract_file_content(change):
    global index, chunk_map
    chunk_map = []

    uploaded_files = file_uploader.value
    if not uploaded_files:
        return

    file_info = uploaded_files[0]
    file_name = file_info["name"]
    file_type = file_name.split(".")[-1].lower()
    file_data = file_info["content"]

    if file_type == "pdf":
        reader = PyPDF2.PdfReader(io.BytesIO(file_data))
        text = "\n".join(
            page.extract_text() for page in reader.pages if page.extract_text()
        )
    elif file_type in ["xlsx", "xls"]:
        df = pd.read_excel(io.BytesIO(file_data))
        text = df.to_csv(index=False)
    else:
        text = "❌ Unsupported file type."

    # Split into chunks
    docs = splitter.split_documents([Document(page_content=text)])

    # Embed and build vector store
    texts = [doc.page_content for doc in docs]
    chunk_map = texts  # Keep for retrieval
    embeddings = embedder.encode(texts, convert_to_numpy=True)

    # Build FAISS index
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(embeddings)

    with output:
        clear_output()
        print(f"✅ File '{file_name}' loaded — {len(texts)} chunks embedded.")
        print("📄 You can now ask questions about the file.")


file_uploader.observe(extract_file_content, names="value")


# --- Q&A HANDLER ---


def on_button_click(b):
    global index, chunk_map

    query = question_input.value
    if not query.strip():
        with output:
            print("⚠️ Please type a question.")
        return

    if index is None or not chunk_map:
        with output:
            print("⚠️ Please upload and process a file first.")
        return

    # Embed the question
    query_embedding = embedder.encode([query], convert_to_numpy=True)

    # Search top 3 similar chunks
    k = 3
    D, I = index.search(query_embedding, k)

    # Build context from top chunks
    top_chunks = [chunk_map[i] for i in I[0]]
    context = "\n---\n".join(top_chunks)

    # Send to local LLM
    prompt = f"You are reading a document. Here are some parts of it:\n\n{context}\n\nNow answer this question:\n{query}"
    response = llm(prompt)

    with output:
        print(f"\n🧠 Question: {query}")
        print(f"📚 Retrieved {k} relevant chunks.")
        print(f"🤖 Answer:\n{response}")

    question_input.value = ""


submit_button.on_click(on_button_click)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\DEANESH\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\DEANESH\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

FileUpload(value=(), accept='.pdf,.xlsx,.xls', description='📁 Upload File')

Output()

In [ ]:
import ipywidgets as widgets
from IPython.display import display

upload = widgets.FileUpload(accept=".pdf,.xlsx,.xls", multiple=False)
display(upload)


def check_upload(change):
    print("Uploader value:", upload.value)


upload.observe(check_upload, names="value")

FileUpload(value=(), accept='.pdf,.xlsx,.xls', description='Upload')